# LSTM for Stock Prediction

https://medium.com/@aniruddha.choudhury94/stock-market-prediction-by-recurrent-neural-network-on-lstm-model-56de700bff68

In [4]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [5]:
dataset = pd.read_csv('data/GENTING_historical_prices.csv',index_col="Date",parse_dates=True)

            Price   Open   High    Low    Vol. Change %
Date                                                   
2019-01-18   6.79   6.83   6.89   6.75  10.25M   -0.44%
2019-01-17   6.82   6.47   6.84   6.47  12.36M    6.23%
2019-01-16   6.42   6.48   6.50   6.38   6.92M   -0.47%
2019-01-15   6.45   6.39   6.48   6.38   4.39M    1.10%
2019-01-14   6.38   6.45   6.49   6.31   3.68M   -1.09%
2019-01-11   6.45   6.43   6.45   6.36   2.73M    0.78%
2019-01-10   6.40   6.47   6.48   6.25   4.06M   -0.16%
2019-01-09   6.41   6.34   6.44   6.25   5.76M    1.58%
2019-01-08   6.31   6.09   6.34   6.07   5.10M    4.13%
2019-01-07   6.06   5.95   6.08   5.95   4.65M    2.19%
2019-01-04   5.93   5.96   5.97   5.91   8.62M   -0.50%
2019-01-03   5.96   6.00   6.06   5.95   3.51M   -0.67%
2019-01-02   6.00   6.12   6.12   6.00   4.34M   -1.64%
2018-12-31   6.10   6.12   6.14   6.08   3.11M    0.66%
2018-12-28   6.06   6.10   6.10   6.04   2.45M   -0.49%
2018-12-27   6.09   6.07   6.13   6.04   2.62M  